# Appendix E -- SQL for Data Scientists
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/APP_E_SQL_for_Data_Scientists.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Prerequisites:** Chapter 3 (NumPy and Pandas)  

### Learning Objectives

- Create and query a SQLite database from Python using `sqlite3`
- Use `pandas.read_sql()` to pull query results directly into DataFrames
- Write aggregation queries: GROUP BY, HAVING, window functions
- Perform JOIN operations and understand when to use SQL vs Pandas
- Load the SO 2025 dataset into SQLite and answer analytical questions with SQL


---

## Setup


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 110

DATASET_URL = 'https://raw.githubusercontent.com/timothy-watt/python-for-ai-ml/main/data/so_survey_2025_curated.csv'
DB_PATH     = '/tmp/so2025.db'

# Load SO 2025 and create SQLite database
df_raw = pd.read_csv(DATASET_URL)
df = df_raw.copy()
df = df.dropna(subset=['ConvertedCompYearly'])
df['ConvertedCompYearly'] = pd.to_numeric(df['ConvertedCompYearly'], errors='coerce')
Q1, Q3 = df['ConvertedCompYearly'].quantile([0.25, 0.75])
IQR = Q3 - Q1
df = df[
    (df['ConvertedCompYearly'] >= max(Q1 - 3*IQR, 5_000)) &
    (df['ConvertedCompYearly'] <= min(Q3 + 3*IQR, 600_000))
].copy()
if 'YearsCodePro' in df.columns:
    df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')
df['uses_python'] = df.get('LanguageHaveWorkedWith', pd.Series(dtype=str)).str.contains('Python', na=False).astype(int)
df['log_salary']  = np.log(df['ConvertedCompYearly'])
df['primary_role'] = df.get('DevType', pd.Series(dtype=str)).str.split(';').str[0].str.strip()
df = df.reset_index(drop=True)

# Write to SQLite
conn = sqlite3.connect(DB_PATH)
keep_cols = [c for c in ['Country', 'EdLevel', 'RemoteWork', 'YearsCodePro',
                          'ConvertedCompYearly', 'log_salary',
                          'uses_python', 'primary_role'] if c in df.columns]
df[keep_cols].to_sql('developers', conn, if_exists='replace', index=False)
print(f'Loaded {len(df):,} rows into SQLite table: developers')
print(f'Columns: {keep_cols}')


---

## E.1 -- Core SQL: SELECT, WHERE, GROUP BY


In [ ]:
# E.1.1 -- Basic SELECT and aggregation

queries = {
    'Count by country (top 10)': """
        SELECT Country,
               COUNT(*)                          AS respondents,
               ROUND(AVG(ConvertedCompYearly), 0) AS avg_salary,
               ROUND(MIN(ConvertedCompYearly), 0) AS min_salary,
               ROUND(MAX(ConvertedCompYearly), 0) AS max_salary
        FROM   developers
        WHERE  Country IS NOT NULL
        GROUP  BY Country
        HAVING COUNT(*) >= 50
        ORDER  BY avg_salary DESC
        LIMIT  10
    """,
    'Python premium by education': """
        SELECT EdLevel,
               uses_python,
               COUNT(*)                          AS n,
               ROUND(AVG(ConvertedCompYearly), 0) AS avg_salary
        FROM   developers
        WHERE  EdLevel IS NOT NULL
        GROUP  BY EdLevel, uses_python
        ORDER  BY EdLevel, uses_python
    """,
}

for title, sql in queries.items():
    print(f'=== {title} ===')
    result = pd.read_sql(sql, conn)
    print(result.to_string(index=False))
    print()


In [ ]:
# E.1.2 -- Window functions: rank within group
# Window functions compute a value for each row relative to a group
# without collapsing the rows (unlike GROUP BY)

sql_window = """
    SELECT Country,
           primary_role,
           ROUND(AVG(ConvertedCompYearly), 0) AS avg_salary,
           COUNT(*)                           AS n,
           RANK() OVER (
               PARTITION BY Country
               ORDER BY AVG(ConvertedCompYearly) DESC
           ) AS rank_in_country
    FROM   developers
    WHERE  Country IN ('United States of America', 'United Kingdom of Great Britain and Northern Ireland',
                       'Germany', 'India')
      AND  primary_role IS NOT NULL
    GROUP  BY Country, primary_role
    HAVING COUNT(*) >= 10
    ORDER  BY Country, rank_in_country
"""

window_df = pd.read_sql(sql_window, conn)
# Show top 3 roles per country
top3 = window_df[window_df['rank_in_country'] <= 3]
print('Top 3 highest-paying roles per country (SQL RANK window function):')
print(top3.to_string(index=False))


In [ ]:
# E.1.3 -- SQL vs Pandas: when to use each

# The same query in both -- compare syntax and output

# SQL version
sql_agg = """
    SELECT   Country,
             COUNT(*)                           AS n,
             ROUND(AVG(ConvertedCompYearly), 0) AS avg_salary,
             ROUND(AVG(uses_python) * 100, 1)   AS pct_python
    FROM     developers
    WHERE    Country IS NOT NULL
    GROUP BY Country
    HAVING   COUNT(*) >= 100
    ORDER BY avg_salary DESC
    LIMIT    8
"""
sql_result = pd.read_sql(sql_agg, conn)

# Equivalent Pandas version
pandas_result = (
    df[df['Country'].notna()]
    .groupby('Country')
    .agg(
        n=('ConvertedCompYearly', 'count'),
        avg_salary=('ConvertedCompYearly', 'mean'),
        pct_python=('uses_python', 'mean')
    )
    .query('n >= 100')
    .sort_values('avg_salary', ascending=False)
    .head(8)
    .round({'avg_salary': 0, 'pct_python': 3})
    .reset_index()
)
pandas_result['pct_python'] = (pandas_result['pct_python'] * 100).round(1)

print('SQL result:')
print(sql_result.to_string(index=False))
print()
print('Pandas result (equivalent):')
print(pandas_result.to_string(index=False))

print()
print('When to use SQL:')
print('  - Data lives in a database (most production data does)')
print('  - You need only a subset of rows -- SQL filters before loading')
print('  - Complex joins across multiple tables')
print('  - Window functions are concise and readable')
print()
print('When to use Pandas:')
print('  - Data is already in memory / a CSV file')
print('  - Complex reshaping, melting, pivoting')
print('  - Chaining with visualisation or ML pipelines')
print('  - The full dataset fits comfortably in RAM')

conn.close()


---

## E.2 -- SQL Quick Reference

```sql
-- Basic structure
SELECT col1, col2, AGG(col3)
FROM   table
WHERE  condition           -- filter rows BEFORE grouping
GROUP  BY col1, col2
HAVING AGG(col3) > value   -- filter AFTER grouping
ORDER  BY col1 DESC
LIMIT  10;

-- Common aggregate functions
COUNT(*)  COUNT(col)  SUM(col)  AVG(col)  MIN(col)  MAX(col)

-- JOIN types
INNER JOIN  -- only matching rows in both tables
LEFT JOIN   -- all rows from left, NULLs where no match in right
FULL JOIN   -- all rows from both (not in SQLite)

-- Window functions
RANK()       OVER (PARTITION BY col ORDER BY col2)
ROW_NUMBER() OVER (PARTITION BY col ORDER BY col2)
LAG(col, 1)  OVER (ORDER BY date_col)   -- previous row value
SUM(col)     OVER (PARTITION BY col ORDER BY date_col)  -- running total

-- Subqueries
SELECT * FROM table WHERE col IN (SELECT col FROM other_table WHERE ...)

-- CTEs (Common Table Expressions) -- readable alternative to subqueries
WITH ranked AS (
    SELECT *, RANK() OVER (PARTITION BY Country ORDER BY Salary DESC) AS rnk
    FROM developers
)
SELECT * FROM ranked WHERE rnk <= 3;
```

---

*End of Appendix E -- Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
